- Import the table from the bronze layer

In [0]:
df_silver_taxi = spark.sql(
  """

  SELECT *
  FROM workspace.bronze.bronze_yellow_taxi_trips_2025_01;
  """
)

- Check the dataset

In [0]:
df_silver_taxi.display()

- Check the length of the dataset

In [0]:
initial_length = (f'Initial length of the dataset : {df_silver_taxi.count()}')

In [0]:
print(initial_length)

- Check the column names

In [0]:
df_silver_taxi.columns

-  Rename some column names

In [0]:
mapping_column_names = {
  'VendorID' : 'vendor_id',
 'tpep_pickup_datetime': 'pickup_datetime',
 'tpep_dropoff_datetime': 'dropoff_datetime',
  'RatecodeID': 'rate_code_id',
  'PULocationID': 'pu_location_id',
 'DOLocationID': 'do_location_id', 
 'Airport_fee': 'airport_fee',
}

In [0]:
# function to rename the column names
def rename_columns(dataframe, old_column, new_column):
    return dataframe.withColumnRenamed(
        old_column, new_column
    )

In [0]:
# apply the function

for old_col, new_col in mapping_column_names.items():
    df_silver_taxi = rename_columns(dataframe=df_silver_taxi, old_column=old_col, new_column=new_col)

In [0]:
# check the result

df_silver_taxi.columns

In [0]:
df_silver_taxi.display()

- Check the schema of the dataset

In [0]:
df_silver_taxi.printSchema()

In [0]:
df_silver_taxi.display()

- Check for nulls

In [0]:
from pyspark.sql.functions import col, when, count

df_silver_taxi.select([count(when(col(c).isNull(), c)).alias(c) for c in df_silver_taxi.columns]
   ).display()


In [0]:
df_silver_taxi.display()

- Data Checks --> dropoff_datetime >= pickup_datetime

In [0]:
check_times = df_silver_taxi.filter(
  " dropoff_datetime < pickup_datetime"
)

check_times.display()

In [0]:
# keep only  dropoff_datetime >= pickup_datetime

df_silver_taxi = df_silver_taxi.filter(
    " dropoff_datetime >= pickup_datetime"
)

In [0]:
df_silver_taxi.display()

- Check duration = 0, if yes , drop that rows

In [0]:
from pyspark.sql import functions as F

df_silver_taxi = df_silver_taxi.filter(
    F.expr("timestampdiff(SECOND, pickup_datetime, dropoff_datetime) != 0")
)


In [0]:
df_silver_taxi.display()

- Check money and fee cols for <0 values

In [0]:
df_silver_taxi.printSchema()

- Remove rows with passengers_count = 0

In [0]:
df_silver_taxi.filter(
    "passenger_count == 0").count()

In [0]:
df_silver_taxi = df_silver_taxi.filter(
  "passenger_count != 0"
)

In [0]:
df_silver_taxi.display()

- Check columns with numerical-money value <0

In [0]:
from pyspark.sql.functions import col

df_silver_taxi.filter(
  (col("trip_distance") < 0) | (col("fare_amount") < 0) | (col("extra") < 0) | (col("tip_amount") < 0) | (col("total_amount") < 0)
).count()

In [0]:
# trip_distance, fare_amount, extra, tip_amount, total_amount
from pyspark.sql.functions import col

df_silver_taxi.filter(
  (col("trip_distance") < 0) | (col("fare_amount") < 0) | (col("extra") < 0) | (col("tip_amount") < 0) | (col("total_amount") < 0)
).display()

In [0]:
from pyspark.sql.functions import col

df_silver_taxi = df_silver_taxi.filter(
  (col("trip_distance") >= 0) | (col("fare_amount") >= 0) | (col("extra") >= 0) | (col("tip_amount")>= 0) | (col("total_amount") >= 0)
)

In [0]:
df_silver_taxi.display()

- Check Categorical column values

In [0]:
categorical_cols = ['store_and_fwd_flag', 'payment_type']

for col in categorical_cols:
    df_silver_taxi.groupBy(col).count().display()

- Location validity

In [0]:
# pu_location_id / do_location_id
from pyspark.sql.functions import col

df_silver_taxi.filter(
  (col("pu_location_id") <= 0) | (col("do_location_id") <= 0)
).count()

In [0]:
df_silver_taxi.display()

In [0]:
final_length = (f'Initial length of the dataset : {df_silver_taxi.count()}')

In [0]:
print(final_length)

- store it into the silver db

%md
- Create new column 'trip_date' grabbing only the date from the "pickup_date" column

In [0]:
from pyspark.sql.functions import to_date, col

df_silver_taxi = df_silver_taxi.withColumn(
  "trip_date", to_date(col("pickup_datetime"))
)

In [0]:
df_silver_taxi.write.format("delta").mode("overwrite").saveAsTable("silver.yellow_taxi_trips_2025_01")